In [ ]:
import psycopg2
import requests
import json
import googlemaps
import re
import csv
import os
import pandas as pd
import pinyin
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from googlemaps import *
from os.path import join, dirname
from sqlalchemy import create_engine

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

#google api key for GEOCODING and PLACES
API_KEY = input("Google API key:")

'''
## 爬取店家資訊的主程式

cities = ["臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市", "基隆市",
         "新竹市", "嘉義市", "新竹縣", "苗栗縣", "彰化縣", "雲林縣",
         "嘉義縣", "屏東縣", "宜蘭縣", "花蓮縣", "臺東縣", "澎湖縣"]

ids = []

for city in cities:
    geocode_result = gmaps.geocode(city)
    loc = geocode_result[0]['geometry']['location']
    
    for place in gmaps.places_nearby(keyword = "海灘", location = loc, radius = 50000)['results']:
        ids.append(place['place_id'])
        
ids = list(set(ids))
info = []
for each_id in ids:
    info.append(gmaps.place(place_id = each_id, language = 'zh-TW')['result'])
    
final_data = pd.DataFrame.from_dict(info)
    
# 我需要找的東西有：名字、經度、緯度、他在哪一個城鎮

allowed = ['沙灘', '海岸', '遊憩區', '浴場', '海濱', '灣', '海灘']
final_list = list(final_data.to_numpy())

sort_out = []
counter = 0

for line in final_list:
    containing = False
    if line[0][0]['long_name'] == '台灣':
        continue
    for i in allowed:
        if i in line[0][0]['long_name']:
            containing = True
            break
    if containing:
        town = ''
        county = ''
        try:
            for element in line[0]:
                if element['types'][0] == 'administrative_area_level_3':
                    town = element['long_name']
                elif element['types'][0] == 'administrative_area_level_2' or element['types'][0] == 'administrative_area_level_1':
                    county = element['long_name']
                    
            if town == '' or county == '':
                references = []
                get = gmaps.places_nearby(location = [line[5]['location']['lng'], line[5]['location']['lat']], radius = 100)['results']
                references.append(gmaps.place(place_id = get[i]['place_id'], language = 'zh-TW')['result']['address_component'] for i in range(2))
                for reference in references:
                    for ref_element in reference[0]:
                        if ref_element['types'][0] == 'administrative_area_level_3':
                            town = element['long_name']
                        elif ref_element['types'][0] == 'administrative_area_level_2' or ref_element['types'][0] == 'administrative_area_level_1':
                            county = element['long_name']
                        
                    
                
            new_line = [line[0][0]['long_name'], line[5]['location']['lng'], line[5]['location']['lat'],
                        town, county]
            sort_out.append(new_line)
            counter += 1
        except:
            continue
            
#beach_big = list(sort_out)
'''



#確認spots.csv在同一目錄下
#讀取非浪點資料
beach94 = []

with open('./spots.csv', newline='',encoding="utf-8") as csvfile:
    
    rows = csv.reader(csvfile)
    firstRow = True;    
    for row in rows:
        if firstRow == True:
            firstRow = False
            continue
        beach94.append(row)

print("loading spots.csv complete")

#取得浪點LIST
url = 'https://swelleye.com/surf-forecasts/'
resp = requests.get(url)
resp.encoding='utf-8' 
soup = BeautifulSoup(resp.text, "html.parser")

#浪點英文名
spot_list = []
#浪點中文名
chi_list = []

for i in soup.find_all("a", class_="list-link"):
    name_href = i['href']
    chi_name = i.text
    
    name = name_href[12:(len(name_href)-14)]
    spot_list.append(name)
    chi_list.append(chi_name[29:(len(chi_name)-25)])
print("loading surf spot complete")
    
#取得浪點經緯度
lon_list = []
lat_list = []

for spot_name in spot_list:
    spot_url = "https://swelleye.com/surf-spots/" + spot_name + "/#surf-forecast"
    resp = requests.get(spot_url)
    resp.encoding='utf-8' 
    soup = BeautifulSoup(resp.text,"html.parser")
    geo_url = soup.find("iframe", height="570px")['src']
    lat = float( geo_url[(geo_url.find('=')+1):geo_url.find('&')] )
    lon = float( geo_url[(geo_url.find( '=', geo_url.find('=')+1 )+1):geo_url.find('&', geo_url.find('&')+1)] )
    lat_list.append(lat)
    lon_list.append(lon)
print("loading lat, lon complete")
    
#取得浪點所在鄉鎮資料
spot_town = []

key = API_KEY
for i in range(len(spot_list)):
    resp = requests.get('http://api.opencube.tw/location?lat='+str(lat_list[i])+'&lng='+str(lon_list[i])+'&key='+key)
    rjson = json.loads(resp.text)
    spot_town.append(rjson["data"]["district"])
print("loading loc town complete")

#取得縣市list
city_url = "https://api.nlsc.gov.tw/other/ListCounty"
resp = requests.get(city_url)
soup = BeautifulSoup(resp.text,"html.parser")
city_list = []
city_id = []
for i in soup.find_all('countyname'):
    city_list.append(i.text)
for i in soup.find_all('countycode'):
    city_id.append(i.text)

#取得鄉鎮list
town_list = []
town_id = []
town_loc_city = []
for i in range(len(city_id)):
    town_url = "https://api.nlsc.gov.tw/other/ListTown/" + city_id[i]
    resp = requests.get(town_url)
    soup = BeautifulSoup(resp.text,"html.parser")
    for k in soup.find_all('townname'):
        town_list.append(k.text)
    for j in soup.find_all('towncode'):
        town_id.append(j.text)
        town_loc_city.append(city_list[i])
print("loading town list complete")        

In [ ]:
#取得商店資訊

gmaps = googlemaps.Client(key = API_KEY)

shop_name_list = []
shop_spot_list = []
shop_rating_list = []
shop_open_list = []
shop_address_list = []

for i in range(len(spot_list)):
    loc = {'lat':lat_list[i], 'lng':lon_list[i]}
    output = gmaps.places_nearby(keyword="衝浪", location = loc, radius = 500)['results']
    for j in range(len(output)):
        if(j<5):
            try:
                shop_name_list.append(output[j]['name'])
                shop_rating_list.append(output[j]['rating'])
                shop_address_list.append(output[j]['vicinity'])
                if('opening_hours' in output[j].keys()):
                    if('open_now' in output[j]['opening_hours'].keys()):
                        shop_open_list.append(output[j]['opening_hours']['open_now'])
                    else:
                        shop_open_list.append(None)
                else:
                    shop_open_list.append(None)
                shop_spot_list.append(i+1)
            except:
                continue
print("loading shop list complete")

In [ ]:
#取得新聞資料
news_title_list = []
news_url_list = []
news_spot_list = []
news_time_list = []

for i in range(len(chi_list)):
    url = 'https://news.google.com/search?q='+chi_list[i]+'衝浪&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
    resp = requests.get(url)
    resp.encoding='utf-8' 
    soup = BeautifulSoup(resp.text,"html.parser")
    for j in soup.find_all("a", class_="DY5T1d RZIKme")[0:5]:
        
        news_title_list.append(j.text)
        news_spot_list.append(i+1)
        url = 'https://news.google.com'+ j['href'][1:]
        news_url_list.append(url)
        
        resp = requests.get(url)
        resp.encoding='utf-8' 
        if(soup.find("time")):
            k = soup.find("time")
            news_time_list.append(str(k.text).replace(' ', ''))
        else:
            news_time_list.append("")
print("loading news list complete")

In [ ]:
#獲取浪點預報
print("crawling using selelium auto click, please do not click the operating browser or press tab")
driver = webdriver.Chrome(ChromeDriverManager().install())

info_spot_id = []
info_date = []
wave_height = []
wave_period = []
wave_dir = []
wind_speed = []
wind_dir = []
temp = []
sea_temp = []

today = datetime.today()
for i in range(len(spot_list)):
    url = "https://swelleye.com/surf-spots/" + spot_list[i] + "/#surf-forecast"
    driver.get(url)
    driver.maximize_window()
    xf = driver.find_element(By.XPATH, '//*[@id="surf-forecast"]/div[2]/div[1]/iframe')
    driver.switch_to.frame(xf)
    #7天預報
    #print(spot_list[i])
    for j in range(6):
        while (1):
            try:
                driver.find_element(By.XPATH, '//*[@id="forecast"]/div[1]/a[' + str(j+1) + ']/div[1]/div[1]').click()
                break
            except:
                print("position failed, auto retry")
        #12個時段
        for k in range(11):
            #浪點
            info_spot_id.append(i+1)
            pred_day = today + timedelta(days=j)
            info_date.append(pred_day.strftime('%Y-%m-%d') + f" {k*2:02d}")
            #浪高(m)
            wave_height.append(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[2]/div[' + str(k+1) + ']/span[1]').text)
            #浪週期(s)
            wave_period.append(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[3]/span[' + str(k+1) + ']').text)
            #浪向(。)
            wave_dir.append( re.sub("[^0-9]", "",str(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[4]/div[' + str(k+1) + ']/div/div').get_attribute('class'))) )
            #風力(m/s)
            wind_speed.append(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[5]/div[' + str(k+1) + ']/span[1]').text)
            #風向(。)
            wind_dir.append( re.sub("[^0-9]", "",str(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[6]/div[' + str(k+1) + ']/div/div').get_attribute('class'))) )
            #溫度(。C)
            temp.append(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[9]/div[' + str(k+1) + ']/span[1]').text)
            #海溫(。C)
            sea_temp.append(driver.find_element(By.XPATH, '//*[@id="forecast-content"]/div[' + str(j+1) + ']/div/div[10]/div[' + str(k+1) + ']/span[1]').text)   
    driver.switch_to.default_content()
driver.quit()
print("loading info list complete")

In [ ]:

#Town Table
df_town = pd.DataFrame(list(zip(town_id, town_list,town_loc_city)), columns =['Id','Name', 'City'])
df_town['Name'] = df_town['Name'].str.replace('臺','台')
df_town['City'] = df_town['City'].str.replace('臺','台')

#Type Table
df_type = pd.DataFrame([[1, '浪點'], [2, '海水浴場'], [3, '海灘'], [4, '其他']], columns = ['Id', 'Name'])

#Spot Table
type_list = [1]*len(chi_list)
spot_town_id = []
other_town_id = []
for i in spot_town:
    if i is None:
        spot_town_id.append(None)
    else:
        spot_town_id.append(df_town.loc[df_town.Name == i, 'Id'].values[0])

df_other = pd.DataFrame(beach94, columns = ['Name', 'Lon', 'Lat', 'Town', 'County', 'index'])
for i in df_other.itertuples(index=False):
    value_id = df_town.loc[df_town.Name == i.Town, 'Id'].values
    if(len(value_id) == 0):
        other_town_id.append(None)
    elif(len(value_id) == 1):
        other_town_id.append(value_id[0])
    else:
        found = False
        for j in value_id:
            if(df_town.loc[df_town.Id == j, 'City'].values[0] == i.County):
                other_town_id.append(j)
                found = True
                break
        if(found == False):
            other_town_id.append(None)
type_other = []
for i in beach94:
    if i:
        if '灘' in i[0]:
            type_other.append(3)
        elif '浴場' in i[0]:
            type_other.append(2)
        else:
            type_other.append(4)
df_other['Town_id'] = other_town_id
df_other['Type_id'] = type_other

df_spot = pd.DataFrame(list(zip(chi_list, lon_list,lat_list,spot_town_id, type_list)), columns =['Name', 'Lon', 'Lat', 'Town_id', 'Type_id'])
df_other2 = df_other[['Name', 'Lon', 'Lat', 'Town_id', 'Type_id']]
df_spot = df_spot.append(df_other2, ignore_index = True)
df_spot['Id'] = df_spot.index+1

#Surfshop Table
df_surfshop = pd.DataFrame(list(zip(shop_name_list, shop_address_list,shop_spot_list,shop_rating_list, shop_open_list)), columns =['Name', 'Address', 'Spot_id', 'Rating', 'Operating_now'])
df_surfshop['Id'] = df_surfshop.index+1

#News Table
df_news = pd.DataFrame(list(zip(news_time_list, news_url_list,news_title_list,news_spot_list)), columns =['Date', 'Url', 'Title', 'Spot_id'])
df_news['Id'] = df_news.index+1

#Information Table
df_information = pd.DataFrame(list(zip(info_spot_id, info_date,
                                       wave_height, wave_period, wave_dir,
                                       wind_speed, wind_dir,
                                       temp, sea_temp)), columns =['Spot_id', 'Date', 'Wave_height', 'Wave_period', 'Wave_direction', 'Wind_speed', 'Wind_direction', 'Temperature', 'Sea_temperature'])

In [ ]:
# 利用 Google API 進行評論爬蟲

total_reviewers = []
total_reviews = []

for place in df_spot.itertuples(index=False):
    geocode_result = gmaps.geocode(place.Name)
    loc = geocode_result[0]['place_id']
    try:
        reference_review = gmaps.place(place_id = loc, language = 'zh-TW')['result']['reviews']
        for i in range(3):
            try:
                review = []
                reviewer = []
                review.append(place.Id)
                review.append(reference_review[i]['author_name'])
                review.append(reference_review[i]['rating'])
                review.append(reference_review[i]['text'])
                review.append(reference_review[i]['time'])

                reviewer.append(reference_review[i]['author_name'])
                reviewer.append(pinyin.get(reference_review[i]['author_name'], format = "numerical") + "@gmail.com")
                total_reviewers.append(reviewer)
                total_reviews.append(review)
            except:
                break
    except:
        continue
print("loading review & reviewer list complete")

In [ ]:
#Reviewer Table
df_reviewer = pd.DataFrame(total_reviewers, columns =['Name', 'Email'])
df_reviewer = df_reviewer.drop_duplicates()
df_reviewer['Id'] = df_reviewer.index+1

#Recommend Table
df_recommend = pd.DataFrame(total_reviews, columns =['Spot_id', 'Reviewer', 'Score', 'Content', 'Date'])
rec_reviewer_id = []
for i in df_recommend.Reviewer:
    rec_reviewer_id.append(df_reviewer.loc[df_reviewer.Name == i, 'Id'].values[0])
df_recommend['Reviewer_id'] = rec_reviewer_id
rec_timestamp = df_recommend['Date']
rec_dt = map(lambda x: (datetime.fromtimestamp(x)).strftime('%Y-%m-%d'), rec_timestamp)
df_recommend['Date'] = list(rec_dt)
df_recommend.drop(columns=["Reviewer"], inplace = True)

In [ ]:
# fill in your db account info
username = input("Input Username:")
password = input("InputPassword:")

conn = psycopg2.connect(
   database="postgres", user=username, password=password, host='localhost', port= '5432'
)
conn.autocommit = True
cursor = conn.cursor()
sql = 'CREATE DATABASE "SURF"';
cursor.execute(sql)
print("Database created successfully........")
conn.close()
conn = psycopg2.connect(
   database="SURF", user=username, password=password, host='localhost', port= '5432'
)
conn.autocommit = True
cursor = conn.cursor()
sql = 'CREATE SCHEMA IF NOT EXISTS "SURF"';
cursor.execute(sql)
print("Schema created successfully........")
conn.close()



engine = create_engine("postgresql://" + username + ":" + password + "@localhost:5432/SURF")
df_type.to_sql('TYPE', engine, schema = 'SURF', if_exists="append", index = False)
df_town.to_sql('TOWN', engine, schema = 'SURF', if_exists="append", index = False)
df_spot.to_sql('SPOT', engine, schema = 'SURF', if_exists="append", index = False)
df_surfshop.to_sql('SURFSHOP', engine, schema = 'SURF', if_exists="append", index = False)
df_information.to_sql('INFORMATION', engine, schema = 'SURF', if_exists="append", index = False)
df_news.to_sql('NEWS', engine, schema = 'SURF', if_exists="append", index = False)
df_reviewer.to_sql('REVIEWER', engine, schema = 'SURF', if_exists="append", index = False)
df_recommend.to_sql('RECOMMEND', engine, schema = 'SURF', if_exists="append", index = False)

print("appending data complete")